<a href="https://colab.research.google.com/github/Existanze54/sirius-neural-networks-2024/blob/main/Practices/S12_LLM_LegNet/LegNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Собираем свой LegNET с SEблоком и конволюциями. Учим LegNet предсказывать экспрессию в клеточной линии HepG2


In [ ]:
!wget https://kodomo.fbb.msu.ru/FBB/ml/vikram_hse_dataset.csv

--2024-05-23 06:33:07--  https://kodomo.fbb.msu.ru/FBB/ml/vikram_hse_dataset.csv
Resolving kodomo.fbb.msu.ru (kodomo.fbb.msu.ru)... 93.180.63.127
Connecting to kodomo.fbb.msu.ru (kodomo.fbb.msu.ru)|93.180.63.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23610749 (23M) [text/csv]
Saving to: ‘vikram_hse_dataset.csv’

vikram_hse_dataset. 100%[===================>]  22.52M  13.4MB/s    in 1.7s    

2024-05-23 06:33:10 (13.4 MB/s) - ‘vikram_hse_dataset.csv’ saved [23610749/23610749]



In [ ]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

## Постановка задачи
<img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/penzar.png?raw=true" width="1200px">

Датасет состоял из 6кк последовательностей по 80 нуклеотидов. Последовательности вставлялись в дрожжи между зоной константно экспрессирующегосся красного белка и белком Reporter, который светится желтым. Далее по экспрессии белков и светимости эти дрожжи распределялись в 18 бинов, вероятности попадания в которые и надо было предсказать

## Архитектура NewLegNET
<img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/FullScheme.png?raw=true" width="1400px">

После Dream Challenge 2022 прошло время, команда попробовала применить свою архитектуру к другим типам задач. В новой архитектуре мы не будем предсказывать распределение по бинам, а сразу перейдем непосредственно к уровню экспрессии той или иной конфигурации региона


In [ ]:
import pandas as pd
vikram_dataset = pd.read_csv("vikram_hse_dataset.csv", sep="\t")

In [ ]:
vikram_dataset.head()

,seq,score
0,AGGACCGGATCAACTAGTATGAGGAGGGTTGTGGAGTGGAAGTGAA...,-0.3105
1,AGGACCGGATCAACTTTGCTCCTTAACACAGGCTAAGGACCAGCTT...,0.9455
2,AGGACCGGATCAACTTCCTGGCCCCCAAGAGCACAGGGAGGCCTTG...,-0.0855
3,AGGACCGGATCAACTTTTTAGACTGTGCAGTTAAGTAAGGCCTCAC...,-0.6655
4,AGGACCGGATCAACTTCCACCGCGCTTCCTCCACGAAACCACACAG...,-0.0135


In [ ]:
vikram_dataset.seq[1]

'AGGACCGGATCAACTTTGCTCCTTAACACAGGCTAAGGACCAGCTTCTTTGGGAGAGAACAGACGCAGGGGCGGGAGGGAAAAAGGGAGAGGCAGACGTCACTTCCTCTTGGCGACTCTGGCAGCAGATTGGTCGGTTGAGTGGCAGAAAGGCAGACGGGGACTGGGCAAGGCACTGTCGGTGACATCACGGACAGGGCGACTTCTATGTAGATGCATTGCGTGAACCGA'

In [ ]:
from torch.utils.data import Dataset

class SeqDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx, :]
        return row.seq, row.score

In [ ]:
import torch.nn.functional as F
def seq2onehot(seq, code={"A": 0, "C": 1, "G": 2, "T": 3}):
    seq = torch.LongTensor([code[s] for s in seq])
    code = F.one_hot(seq, num_classes=4)
    return code.transpose(0, 1).float()

In [ ]:
print(seq2onehot("ACGT"))

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [ ]:
from torch.utils.data import Dataset

class SeqDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx, :]
        return seq2onehot(row.seq), row.score

In [ ]:
SeqDataset(vikram_dataset)[0][0].dtype

torch.float32

In [ ]:
SeqDataset(vikram_dataset)[0][0]

tensor([[1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
         0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 0., 0.,

# Задание - воссоздать архитектуру NewLegNET


## Stem block
<img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/StemBlock.png?raw=true" width="600px">

### Eff block - небольшая модификация EfficientNetV2

<img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/EffBlockk.png?raw=true" width="400px">

Параметр groups передается непосредственно в поле Conv1D и равен размерности хиден слоя

###  Squeeze and Excitation (SE) block
<img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/SEBlock.png?raw=true" width="400px">

Красным обозначены слои, которые убрали в новой версии LegNET. Тем не менее, слой multiplication (умножение аутпута сигмоиды на инпут всего блока) мы оставляем

## Обзор всех блоков
<img src="https://github.com/Existanze54/sirius-neural-networks-2024/blob/main/Images/OurLegNET.png?raw=true" width="800px">

Игнорируем часть D., ее не надо трогать в архитектуре


# А дальше вы сами)))

ну почти

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

def initialize_weights(m):
    if isinstance(m, nn.Conv1d):
        n = m.kernel_size[0] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2 / n))
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_(0, 0.001)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)

class SELayer(nn.Module):
    def __init__(self, inp, reduction=4): # reduction - значение, на которое как раз делится количество каналов
        # your code here

    def forward(self, x):
        b, c, _, = x.size() # b - номер батча, c -  номер канала
        y = x.view(b, c, -1).mean(dim=2) # некоторый аналог global average pooling из предыдущей архитектуры
        y = # your code here
        # не забудьте привести x и y к одному виду для умножения
        return x * y

class EffBlock(nn.Module):
    def __init__(self, in_ch, ks, resize_factor, activation, out_ch=None, se_reduction=None):
        super().__init__()
        self.in_ch = in_ch
        self.out_ch = self.in_ch if out_ch is None else out_ch
        self.resize_factor = resize_factor # увеличение числа каналов
        self.se_reduction = resize_factor if se_reduction is None else se_reduction # уменьшение для SE блока
        self.ks = ks  # размер ядра свертки
        self.inner_dim = self.in_ch * self.resize_factor # размер хидден слоя

        # your code here

    def forward(self, x):
        # your code here

class StemBlock(nn.Module):
    def __init__(self, in_ch, ks, activation, out_ch=None):
        super().__init__()
        self.in_ch = in_ch
        self.out_ch = self.in_ch if out_ch is None else out_ch
        self.ks = ks

        # your code here

    def forward(self, x):
         # your code here

class ResidualConcat(nn.Module): # оставляем как есть
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return torch.concat([self.fn(x, **kwargs), x], dim=1)

class MapperBlock(nn.Module): # оставляем как есть
    def __init__(self, in_features, out_features, activation=nn.SiLU):
        super().__init__()
        self.block = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Conv1d(in_channels=in_features,
                      out_channels=out_features,
                      kernel_size=1),
        )

    def forward(self, x):
        return self.block(x)

class HumanLegNet(nn.Module):
    def __init__(self,
                 in_ch,
                 stem_ch,
                 stem_ks,
                 ef_ks,
                 ef_block_sizes,
                 pool_sizes,
                 resize_factor,
                 activation=nn.SiLU,
                 ):
        super().__init__()
        assert len(pool_sizes) == len(ef_block_sizes)

        self.in_ch = in_ch
        self.stem = # StemBlock()

        blocks = []

        in_ch = stem_ch
        out_ch = stem_ch
        for pool_sz, out_ch in zip(pool_sizes, ef_block_sizes):
            blc = nn.Sequential(
                ResidualConcat(# EffBlock()
                               ),
                # StemBlock(),
                nn.MaxPool1d(pool_sz) if pool_sz != 1 else nn.Identity()
            )
            in_ch = out_ch
            blocks.append(blc)
        self.main = nn.Sequential(*blocks)

        self.mapper = MapperBlock(in_features=out_ch,
                                  out_features=out_ch * 2)
        self.head = nn.Sequential(nn.Linear(out_ch * 2, out_ch * 2),
                                   nn.BatchNorm1d(out_ch * 2),
                                   activation(),
                                   nn.Linear(out_ch * 2, 1)) # на выход получили одну чиселку

    def forward(self, x):
        x = self.stem(x)
        x = self.main(x)
        x = self.mapper(x)
        x = F.adaptive_avg_pool1d(x, 1)
        x = x.squeeze(-1)
        x = self.head(x)
        x = x.squeeze(-1)
        return x


In [ ]:
import pytorch_lightning as L
from torchmetrics import MeanSquaredError

class SeqModel(L.LightningModule):
    def __init__(self, lr=3e-4, onecycle=False):
        super().__init__()
        self.model = HumanLegNet(in_ch=4,
                                 stem_ch=64,
                                 stem_ks=11,
                                 ef_ks=9,
                                 ef_block_sizes=[80, 96, 112, 128],
                                 pool_sizes=[2,2,2,2],
                                 resize_factor=4) # создаем нашу модель
        self.criterion = nn.MSELoss() # задали лосс
        self.lr = lr
        self.val_mse = MeanSquaredError() # метрика тоже по МСЕ считается
        self.onecycle = onecycle # можно и тру поставить, фил фри так сказать
        # пишем, какую метрику нам интересно смотреть на датасете
    def forward(self, x):
        # пишем, как работать с изображением
        return self.model(x)

    def training_step(self, batch, batch_nb):
        # описываем один шаг тренировки
        x, y = batch
        y = y.float()
        loss = self.criterion(self(x), y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self): # алгоритм OneCycle
        if self.onecycle: # используем OneCycle
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr / 50)
            lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                            total_steps=self.trainer.estimated_stepping_batches,
                                                            max_lr=0.01)
            lr_scheduler_config = {
                    # REQUIRED: The scheduler instance
                    "scheduler": lr_scheduler,
                    # The unit of the scheduler's step size, could also be 'step'.
                    # 'epoch' updates the scheduler on epoch end whereas 'step'
                    # updates it after a optimizer update.
                    "interval": "step",
                    # How many epochs/steps should pass between calls to
                    # `scheduler.step()`. 1 corresponds to updating the learning
                    # rate after every epoch/step.
                "frequency": 1,
                    # If using the `LearningRateMonitor` callback to monitor the
                    # learning rate progress, this keyword can be used to specify
                    # a custom logged name
                    "name": "cycle_lr"
            }
            return [optimizer], [lr_scheduler_config]
        else: # просто учим
            return torch.optim.AdamW(self.parameters(), lr=self.lr)

    def validation_step(self, batch, batch_idx):
        # пишем то, как осуществляется шаг валидации
        x, y = batch
        y = y.float()
        preds = self(x)
        self.val_mse.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_acc", self.val_mse, prog_bar=True)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if isinstance(batch, tuple) or isinstance(batch, list):
            x, _ = batch
        else:
            x = batch
        return self(x)

In [ ]:
import pandas as pd
vikram_dataset = pd.read_csv("vikram_hse_dataset.csv", sep="\t")

In [ ]:
vikram_dataset.head()

,seq,score
0,AGGACCGGATCAACTAGTATGAGGAGGGTTGTGGAGTGGAAGTGAA...,-0.3105
1,AGGACCGGATCAACTTTGCTCCTTAACACAGGCTAAGGACCAGCTT...,0.9455
2,AGGACCGGATCAACTTCCTGGCCCCCAAGAGCACAGGGAGGCCTTG...,-0.0855
3,AGGACCGGATCAACTTTTTAGACTGTGCAGTTAAGTAAGGCCTCAC...,-0.6655
4,AGGACCGGATCAACTTCCACCGCGCTTCCTCCACGAAACCACACAG...,-0.0135


## Обучение
Делим на трейн, тест и вал датасеты

In [ ]:
from torch.utils.data import DataLoader, random_split

dataset = SeqDataset(vikram_dataset)
test_size = int(0.2 * len(dataset))
remain_size = len(dataset) - test_size
val_size = int(0.2 * remain_size)
train_size = remain_size - val_size
remain_dataset, test_dataset = random_split(dataset, [remain_size, test_size])
train_dataset, val_dataset = random_split(remain_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(62933, 15733, 19666)

# Запускаем

In [ ]:
model = SeqModel(lr=3e-4, onecycle=True)

trainer = L.Trainer(max_epochs=10)  # gpus=1 if torch.cuda.is_available() else 0

trainer.fit(model, train_loader)



# Фиксируем результат

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32)
trainer.test(model, test_loader)